In [4]:
import requests
import pandas as pd
import json

In [2]:
import getpass
rapidAPIkey = getpass.getpass()

········


Following the class example, create a function that returns the price, names of origin and arrival airports and the name of the company. Do it for all the flights between two dates.

In [10]:
def city_code(x):
    url = "https://skyscanner-skyscanner-flight-search-v1.p.rapidapi.com/apiservices/autosuggest/v1.0/UK/GBP/en-GB/"
    params = {"query":x}
    headers = {'x-rapidapi-host': "skyscanner-skyscanner-flight-search-v1.p.rapidapi.com",
           'x-rapidapi-key': rapidAPIkey}
    response = requests.get(url, headers = headers, params = params)
    
    return response.json()['Places'][0]['CityId']

In [126]:
def flight_prices(departure, arrival, date):
    d = city_code(departure)
    a = city_code(arrival)
    
    url = f"https://skyscanner-skyscanner-flight-search-v1.p.rapidapi.com/apiservices/browsequotes/v1.0/US/USD/en-US/{d}/{a}/{date}"

    params = {"inboundpartialdate":date}

    headers = {
        'x-rapidapi-host': "skyscanner-skyscanner-flight-search-v1.p.rapidapi.com",
        'x-rapidapi-key': rapidAPIkey}

    response = requests.get(url, headers=headers, params=params)
    x=pd.json_normalize(response.json())
    
    quotes = pd.DataFrame(x["Quotes"][0])
    quotes.set_index(quotes.columns[0], inplace=True)

    carriers = pd.DataFrame(x["Carriers"][0])
    carriers.set_index(carriers.columns[0], inplace=True)

    places = pd.DataFrame(x["Places"][0])
    places.set_index(["PlaceId"], inplace=True)
    
    df = pd.DataFrame(columns=['date','price','origin_airport','arrival_airport','airline'])
    
    for i in range(1,len(quotes)+1):
        p = quotes.MinPrice[i]
        oa = places.Name[quotes.OutboundLeg[i]['OriginId']]
        aa = places.Name[quotes.OutboundLeg[i]['DestinationId']]
        al = carriers.Name[quotes.OutboundLeg[i]['CarrierIds'][0]]
        df.loc[df.shape[0]] = [date, p, oa, aa, al]
    
    return df

In [12]:
def get_dates(start, end):
    return pd.Series(pd.date_range(start, end,freq='d').format())

In [137]:
def flight_options(origin, destination, start, end):
    dates = get_dates(start, end)
    df = pd.DataFrame(columns=['date','price','origin_airport','arrival_airport','airline'])
    
    for date in dates:
        x = flight_prices(origin, destination, date)
        df = pd.concat([df, x], axis=0)
    df.reset_index(drop=True, inplace=True)
    return df

In [138]:
flight_options('Madrid', 'Barcelona', '2021-10-19', '2021-10-20' )

,date,price,origin_airport,arrival_airport,airline
0,2021-10-19,28,Madrid,Barcelona,Ryanair
1,2021-10-19,29,Madrid,Barcelona,Vueling Airlines
2,2021-10-20,26,Madrid,Barcelona,Ryanair
3,2021-10-20,29,Madrid,Barcelona,Vueling Airlines
